In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data creation


In [2]:
class Activity:
    def __init__(self, name, id):
        self.name = name
        self.id = id

    def __print__(self):
        print(f"ProcessID {self.id}: {self.name}")

In [3]:
class Process:
    def __init__(self, name, id, activities):
        self.name = name
        self.id = id
        self.activities = activities

    def __print__(self):
        print(f'Process {self.name} consists of following steps (ordered): \n')
        for a in self.activities:
            print(a.__print__())


In [4]:
class ProcessList:
    def __init__(self, name, processes):
        self.name = name
        self.processList = processes
        self.sequenceList = self.fillSequenceList()
        self.uniqueActivities = self.getUniqueActivities()
        self.initActivities = self.getInitActivities()
        self.finalActivities = self.getFinalActivities()
        self.FootprintMatrix = None
        self.FootprintMatriSymbolic = None
        self.maximalPairs = None

    
    def fillSequenceList(self):
        sequenceList = []
        for process in self.processList:
            sequence = []
            for activity in process.activities:
                sequence.append(activity.name)
            sequenceList.append(sequence)
        return sequenceList

    def getUniqueActivities(self):
        T_L = {}
        for process in self.processList:
            for activity in process.activities:
                if activity.name not in T_L:
                    T_L[activity.name] = activity

        T_L = list(T_L.values())
        # sort T_L alphabetically
        T_L.sort(key=lambda x: x.name)
        self.uniqueActivities = T_L
        return T_L
    
    def getInitActivities(self):
        T_I = {}
        for process in self.processList:
            initActivity = process.activities[0]
            if initActivity.name not in T_I:
                T_I[initActivity.name] = initActivity.name
        self.initActivities = list(T_I.values())

        return self.initActivities
    
    def getFinalActivities(self):
        T_O = {}
        for process in self.processList:
            finalActivity = process.activities[-1]
            if finalActivity.name not in T_O:
                T_O[finalActivity.name] = finalActivity.name
        self.finalActivities = list(T_O.values())
        return list(T_O.values())
    #! FIX TO WORK AS LIST OF ACTIVITIES

    def generateFootprintMatrix(self):
        #ToDo Implement
        
        # Define the sequence
        sequence = self.sequenceList

        # Determine the maximum length of the sub-arrays
        max_length = max(len(sub_list) for sub_list in sequence)

        # Create a numpy array with the maximum length and pad with zeros
        # Use empty strings ('') for padding as we are dealing with strings
        padded_sequence = np.array([sub_list + [''] * (max_length - len(sub_list)) for sub_list in sequence])
        sequence = padded_sequence

        allTokens = sequence.flatten()
        allTokens = np.unique(allTokens)

        # print(sequence)
        # print(allTokens )

        FootprintMatrix = np.ones((len(allTokens),len(allTokens)))*4

                # the 4 patterns
                # direct succession     1
                # causality             2
                # parallel              3
                # choice                4

        # fill the matrix
        for s in sequence:
            for i in range(len(s)-1):
                # direct succession     1
                x = s[i]
                y = s[i+1]
                x_index = np.where(allTokens == x)[0][0]
                y_index = np.where(allTokens == y)[0][0]
                FootprintMatrix[x_index][y_index] = 1

                # causality             2
                if FootprintMatrix[y_index][x_index] == 4:
                    FootprintMatrix[y_index][x_index] = 2 

                # parallel              3
                if FootprintMatrix[x_index][y_index] == 2:
                    FootprintMatrix[x_index][y_index] = 3

                # choice                4
                if FootprintMatrix[x_index][y_index] == 0:
                    FootprintMatrix[x_index][y_index] = 4

        # delete first row and column
        FootprintMatrix = np.delete(FootprintMatrix,0,0)
        FootprintMatrix = np.delete(FootprintMatrix,0,1)

        def convertFootprintMatrixToVanAalstNotation(FootprintMatrix):
            FootprintMatrix = np.where(FootprintMatrix == 1, '>', FootprintMatrix)
            FootprintMatrix = np.where(FootprintMatrix == '2.0', '->', FootprintMatrix)
            FootprintMatrix = np.where(FootprintMatrix == 3, '||', FootprintMatrix)
            FootprintMatrix = np.where(FootprintMatrix == '4.0', '#', FootprintMatrix)
            return FootprintMatrix

        self.FootprintMatrix = FootprintMatrix
        FootprintMatrix = convertFootprintMatrixToVanAalstNotation(FootprintMatrix)
        self.FootprintMatriSymbolic = FootprintMatrix
        return FootprintMatrix      

    def getListOfPairs(self):
        # Ensure to delete all non-maximal pairs
        direct_pairs = []

        # Go line by line through the FootprintMatrix
        for i in range(len(self.FootprintMatrix)):
            for j in range(i+1, len(self.FootprintMatrix[i])):
                if self.FootprintMatrix[i][j] == 1:
                    direct_pairs.append((self.uniqueActivities[i].name, self.uniqueActivities[j].name))

        maximal_pairs = set(direct_pairs)

        for (X,Y) in direct_pairs:
            for (A,B) in direct_pairs:
                if (B,Y) in direct_pairs and (X,A) in direct_pairs:
                    if (A, B) != (X, Y):
                        if (X, Y) in maximal_pairs:
                            maximal_pairs.remove((X, Y))

        self.maximalPairs = maximal_pairs

        return maximal_pairs
         


    
    def printAlphaMinerOutput(self):
        # Step 1s
        self.getUniqueActivities()
        # Step 2
        self.getInitActivities()
        # Step 3
        self.getFinalActivities()
        # Step 4
        self.generateFootprintMatrix()
        self.getListOfPairs()



        print("Unique activities found:")
        print(",".join(a.name for a in self.uniqueActivities))
        print("Initial activities found:")
        print(",".join(a for a in self.initActivities))
        print("Final activities found:")
        print(",".join(a for a in self.finalActivities))
        print("Footprint matrix:")
        print(self.FootprintMatrix)
        print("Footprint matrix (symbolic):")
        print(self.FootprintMatriSymbolic)
        print("List of pairs:")
        print(self.getListOfPairs())




    def __print__(self):
        print(f'This process list contains {len(self.processList)} executions of process: {self.name}')

In [5]:
def createData(sequences, path='data/defaultData'):
    data = []
    i = 0
    for sequence in sequences:
        d = []
        for element in sequence:
            d.append(Activity(element, i))
        p = Process("Example", i, d)
        data.append(p)
        i += 1
    return data

In [6]:
# sequence = [['a','c','b'],['b','c','d'],['a','c','e'],['b','c','e']]
sequence = [['a','b','c','d','f'],['a','b','d','c','f'],['a','c','b','d','f'],['a','e','f']]
data = createData(sequence)
print(data)
processesList = ProcessList("Example",data)


[<__main__.Process object at 0x1240673d0>, <__main__.Process object at 0x124067ed0>, <__main__.Process object at 0x1240871d0>, <__main__.Process object at 0x124086e90>]


# Alpha algorithm

[x] 1. Scan the entire event log, check all unique activities.  
 Set $T_L$

[X] 2. Initial activities. Set $T_I$

[X] 3. Find final activities. Set $T_O$

[X] 4. Footprint Matrix
[ ] Calculate pairs

[ ] 5. Delete non-maximal pairs


In [7]:
(processesList.printAlphaMinerOutput())


Unique activities found:
a,b,c,d,e,f
Initial activities found:
a
Final activities found:
f
Footprint matrix:
[[4. 1. 1. 4. 1. 4.]
 [2. 4. 1. 1. 4. 4.]
 [2. 1. 4. 1. 4. 1.]
 [4. 2. 1. 4. 4. 1.]
 [2. 4. 4. 4. 4. 1.]
 [4. 4. 2. 2. 2. 4.]]
Footprint matrix (symbolic):
[['#' '>' '>' '#' '>' '#']
 ['->' '#' '>' '>' '#' '#']
 ['->' '>' '#' '>' '#' '>']
 ['#' '->' '>' '#' '#' '>']
 ['->' '#' '#' '#' '#' '>']
 ['#' '#' '->' '->' '->' '#']]
List of pairs:
{('d', 'f'), ('a', 'e'), ('c', 'd'), ('a', 'c'), ('c', 'f'), ('e', 'f'), ('b', 'd'), ('a', 'b'), ('b', 'c')}


Step 4 -> Create the FootprintMatrix


In [8]:
# # Define the sequence
# sequence = [['a','b','c','d','f'],['a','b','d','c','f'],['a','c','b','d','f'],['a','e','f']]

# # Determine the maximum length of the sub-arrays
# max_length = max(len(sub_list) for sub_list in sequence)

# # Create a numpy array with the maximum length and pad with zeros
# # Use empty strings ('') for padding as we are dealing with strings
# padded_sequence = np.array([sub_list + [''] * (max_length - len(sub_list)) for sub_list in sequence])
# sequence = padded_sequence

# allTokens = sequence.flatten()
# allTokens = np.unique(allTokens)

# print(sequence)
# print(allTokens )

# FootprintMatrix = np.ones((len(allTokens),len(allTokens)))*4

#         # the 4 patterns
#         # direct succession     1
#         # causality             2
#         # parallel              3
#         # choice                4

# # fill the matrix
# for s in sequence:
#     for i in range(len(s)-1):
#         # direct succession     1
#         x = s[i]
#         y = s[i+1]
#         x_index = np.where(allTokens == x)[0][0]
#         y_index = np.where(allTokens == y)[0][0]
#         FootprintMatrix[x_index][y_index] = 1

#         # causality             2
#         if FootprintMatrix[y_index][x_index] == 4:
#             FootprintMatrix[y_index][x_index] = 2 

#         # parallel              3
#         if FootprintMatrix[x_index][y_index] == 2:
#             FootprintMatrix[x_index][y_index] = 3

#         # choice                4
#         if FootprintMatrix[x_index][y_index] == 0:
#             FootprintMatrix[x_index][y_index] = 4

# # delete first row and column
# FootprintMatrix = np.delete(FootprintMatrix,0,0)
# FootprintMatrix = np.delete(FootprintMatrix,0,1)

# def convertFootprintMatrixToVanAalstNotation(FootprintMatrix):
#     FootprintMatrix = np.where(FootprintMatrix == 1, '>', FootprintMatrix)
#     FootprintMatrix = np.where(FootprintMatrix == '2.0', '->', FootprintMatrix)
#     FootprintMatrix = np.where(FootprintMatrix == 3, '||', FootprintMatrix)
#     FootprintMatrix = np.where(FootprintMatrix == '4.0', '#', FootprintMatrix)
#     return FootprintMatrix



# FootprintMatrix = convertFootprintMatrixToVanAalstNotation(FootprintMatrix)
# print(FootprintMatrix)

# print(processesList.generateFootprintMatrix())
